# **Análise e predição do posicionamento político do eleitor brasileiro**

Projeto final realizado para a disciplina de Ciência de Dados do curso de Ciência da Computação, *Universidade de São Paulo*.

## **Introdução**

O presente trabalho tem como objetivo realizar, primeiramente, uma *análise exploratória* dos dados provenientes de uma pesquisa de opinião sobre o perfil ideológico dos eleitores brasileiros, feita pelo Instituto DataSenado no ano de 2024. O conjunto de dados original contém diversas variáveis de natureza sociodemográfica, comportamental e política, mas nem todas serão utilizadas. Serão mantidas somente aquelas que possam contribuir para compreender a variável alvo (*target*) **Posicionamento Político**, que representa o espectro político autodeclarado por cada participante. A análise de dados buscará, portanto, investigar as relações entre a variável target e as demais variáveis selecionadas, por meio de visualizações gráficas e métodos estatísticos. 

Após a fase exploratória, o projeto avançará para a segunda etapa: a construção de um modelo preditivo. Com o uso de técnicas de *machine learning*, o objetivo será desenvolver um modelo capaz de prever o espectro político com base nas respostas às variáveis selecionadas da pesquisa.

Importante mencionar que este trabalho não se preocupará em fazer inferência sobre a preferência política da população brasileira como um todo, tendo em vista que uma parcela significativa dos dados originais não será utilizada.

* <small>Os dados utilizados neste trabalho foram obtidos por meio dos microdados disponibilizados pelo Instituto DataSenado em <a href="https://www12.senado.leg.br/institucional/datasenado/publicacaodatasenado?id=pesquisa-traca-perfil-ideologico-dos-eleitores-brasileiros" target="_blank" rel="noopener noreferrer">Pesquisa traça perfil ideológico dos eleitores brasileiros</a>.</small>
* <small>Junto com os dados foi disponibilizado um dicionário para auxiliar na interpretação.</small>

## **1. Importação de bibliotecas**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## **2. Carregamento dos dados**

In [2]:
df = pd.read_csv(r'data\raw\panorama_politico_06_2024\DATASEN BR 2024 06 BAROMETRO - DADOS.csv', sep=';')

## **3. Visão geral dos dados não tratados**

In [3]:
df.shape

(21808, 109)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21808 entries, 0 to 21807
Columns: 109 entries, ID to W2
dtypes: float64(49), int64(52), object(8)
memory usage: 18.1+ MB


In [5]:
df.head()

,ID,P04,P06,P07,P08,P09,P10,P11,P12,P13,...,VD_EDUCACAO,POP_UF_SEXO,POP_UF_RACA,POP_UF_IDADE,POP_UF_RESIDENCIA,POP_UF_PORTE,POP_UF_TRABALHO,POP_UF_EDU,W1,W2
0,1,41,3,3,4,2,2,2,2,1,...,4,"4599548,01319187","5928889,83043946","1736626,25524565","8334552,9943779",4791536,"5981782,45942209","2337434,74097726","2,79749300778953","12549,6687231227"
1,2,29,2,2,2,2,2,2,2,2,...,3,"6203186,58563845","9446410,89141747","2251115,21205504","8765141,45702551",3033950,"970710,50572401","3842125,61215856","1,12345379164713","2805,47489557209"
2,3,28,1,2,2,2,2,2,2,2,...,3,"966058,41748564","1406439,72866629","527384,65232656","371734,75555842",998303,"995087,55192826","527747,57235541","0,335079450803203","3533,35524432939"
3,4,28,3,2,2,2,1,2,2,2,...,1,"966058,41748564","1406439,72866629","374974,77422564","1462103,48138951",608944,"728396,78367523","679620,91966034","0,335079450803203","16466,2467766673"
4,5,24,1,2,1,3,2,2,2,1,...,3,"1345034,43392631","1067871,54843917","582338,13699172","2344797,51342096",973772,"1378587,88363752","865902,52059461","0,242935105861798","2386,70789580883"


O conjunto de dados inicialmente contém 21.808 linhas e 109 colunas (perguntas). Cada linha é a resposta de um entrevistado para as 109 perguntas. É possível observar que tanto as perguntas quanto as respostas estão em formato numérico. Dessa forma, para facilitar o entendimento e a análise dos dados, cabe converter esses valores para a forma textual.

## **4. Tratamento dos dados**

### **4.1. Filtrando por colunas importantes**

Após análise detalhada do dicionário do conjunto de dados, foi possível selecionar as perguntas mais relevantes para serem correlacionadas com o espectro político (pergunta **P64**). Nestre trabalho, serão incluídas apenas as respostas que expressem uma **posição política clara (esquerda, direita ou centro)**, de tal forma que serão desconsideradas opções neutras ou indefinidas.

In [6]:
selected_columns = [ 
    "P04",
    "P50",
    "P51",
    "P52",
    "P54",
    "P55",
    "P56",
    "P57",
    "P58",
    "P59",
    "P60",
    "P61",
    "P62",
    "P63",
    "V02",
    "V03",
    "V08",
    "V27",
    "VD_EDUCACAO",
    "VD_REGIAO",
    "VD_IDADE",
    "P64"  # Variável alvo que identifica o espectro político
]

df = df[selected_columns]
df = df[df['P64'].isin([1, 2, 3])].reset_index(drop=True) # Para filtrar apenas respostas: esquerda, direita, centro

In [7]:
df.shape

(12345, 22)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12345 entries, 0 to 12344
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   P04          12345 non-null  int64  
 1   P50          8901 non-null   float64
 2   P51          8901 non-null   float64
 3   P52          8901 non-null   float64
 4   P54          8901 non-null   float64
 5   P55          12345 non-null  int64  
 6   P56          12345 non-null  int64  
 7   P57          12345 non-null  int64  
 8   P58          12345 non-null  int64  
 9   P59          12345 non-null  int64  
 10  P60          12345 non-null  int64  
 11  P61          12345 non-null  int64  
 12  P62          12345 non-null  int64  
 13  P63          12345 non-null  int64  
 14  V02          12345 non-null  int64  
 15  V03          12345 non-null  int64  
 16  V08          12345 non-null  int64  
 17  V27          12345 non-null  int64  
 18  VD_EDUCACAO  12345 non-null  int64  
 19  VD_R

### **4.2. Lidando com dados faltantes**

In [9]:
df.isna().sum()

P04               0
P50            3444
P51            3444
P52            3444
P54            3444
P55               0
P56               0
P57               0
P58               0
P59               0
P60               0
P61               0
P62               0
P63               0
V02               0
V03               0
V08               0
V27               0
VD_EDUCACAO       0
VD_REGIAO         0
VD_IDADE          0
P64               0
dtype: int64

In [10]:
df[df[['P50', 'P51', 'P52', 'P54']].isna().all(axis=1)].shape[0]

3444

In [11]:
df.isna().sum() / df.shape[0]

P04            0.000000
P50            0.278979
P51            0.278979
P52            0.278979
P54            0.278979
P55            0.000000
P56            0.000000
P57            0.000000
P58            0.000000
P59            0.000000
P60            0.000000
P61            0.000000
P62            0.000000
P63            0.000000
V02            0.000000
V03            0.000000
V08            0.000000
V27            0.000000
VD_EDUCACAO    0.000000
VD_REGIAO      0.000000
VD_IDADE       0.000000
P64            0.000000
dtype: float64

Como é possível observar, **há 3.444 valores nulos** para as variáveis **P50**, **P51**, **P52** e **P54**, que sempre ocorrem simultaneamente, ou seja, sempre que uma dessas variáveis é nula, todas as outras também o são. Essas ausências, entretanto, não representam dados faltantes no sentido tradicional, mas sim um caso de **ausência por desenho** do questionário (***missing by design***), que refere-se à prática intencional de **omitir dados** durante a coleta para otimizar a pesquisa ou para obter informações específicas, em vez de considerar os dados como perdidos por acidente ou erro. Conforme indicado no questionário, as perguntas P47 a P49 possuem uma lógica de salto: dependendo das respostas dadas pelo entrevistado, o entrevistador **avançará diretamente** para a pergunta P55.

<p style="text-align:center;">
  <img src="images/missing.png" alt="missing" style="max-width:100%; height:auto;">
</p>

Isso representa 3.444 linhas, quase **30% do conjunto de dados**. Assim, para não perder essa quantidade significativa de dados, será adotada a abordagem de substituir os valores faltantes por "*-1 : Não se aplica*".

In [12]:
cols = ['P50', 'P51', 'P52', 'P54']
mask = df[cols].isna().all(axis=1)
df.loc[mask, cols] = -1

df.isna().sum()

P04            0
P50            0
P51            0
P52            0
P54            0
P55            0
P56            0
P57            0
P58            0
P59            0
P60            0
P61            0
P62            0
P63            0
V02            0
V03            0
V08            0
V27            0
VD_EDUCACAO    0
VD_REGIAO      0
VD_IDADE       0
P64            0
dtype: int64

In [13]:
df.shape

(12345, 22)

### **4.3. Lidando com dados duplicados**

In [14]:
df.duplicated().sum()

2

In [15]:
df[df.duplicated(keep=False)]

,P04,P50,P51,P52,P54,P55,P56,P57,P58,P59,...,P62,P63,V02,V03,V08,V27,VD_EDUCACAO,VD_REGIAO,VD_IDADE,P64
1596,16,2.0,2.0,1.0,1.0,1,2,1,2,1,...,3,1,2,3,2,2,4,1,2,2
3754,53,4.0,1.0,1.0,1.0,1,2,1,1,2,...,1,1,2,3,1,2,4,5,3,1
3833,16,2.0,2.0,1.0,1.0,1,2,1,2,1,...,3,1,2,3,2,2,4,1,2,2
3980,53,4.0,1.0,1.0,1.0,1,2,1,1,2,...,1,1,2,3,1,2,4,5,3,1


Conforme os resultados acima, pode-se observar a presença de **dois dados duplicados**. No entanto, **eles não serão removidos**, haja vista que esse conjunto de dados se trata de uma pesquisa realizada com várias pessoas nos diversos estados do país, e que é pouco provável, mas não impossível, que duas pessoas do mesmo estado tenham exatamente as mesmas respostas para as mesmas perguntas.

### **4.4. Convertendo para valores textuais**

Finalmente, pode-se realizar o mapeamento dos valores numéricos para seus respectivos valores textuais, considerando o dicionário de dados disponível. O conjunto de dados tratado em **4.2** não será descartado, pois será utilizado posteriormente para construir o modelo preditivo.

In [16]:
def map_df_to_text(df_to_translate: pd.DataFrame, path_excel: str, columns_names: list[str]) -> pd.DataFrame:
    """
    Função para mapear os valores numéricos do DataFrame para seus equivalentes textuais,
    por meio de um dicionário de dados fornecido em um arquivo Excel.
    O DataFrame original não é modificado, e uma cópia sua traduzida é retornada.

    Parâmetros:
    - df_to_translate: pd.DataFrame
        DataFrame contendo os dados com valores numéricos a serem traduzidos.
    - path_excel: str
        Caminho para o arquivo Excel que contém o dicionário de dados.
    - columns_names: list[str]
        Lista com os novos nomes das colunas para o DataFrame traduzido.
    """
    try:
        # --- Passo 1: Carregar o Dicionário de Dados (arquivo XLSX) ---
        df_excel = pd.read_excel(path_excel)

        # --- Passo 2: Dicionário para VALORES das colunas ---
        # Preenche as células vazias na coluna 'Código da variável' com o valor não nulo mais recente
        df_excel['Código da variável'] = df_excel['Código da variável'].ffill()

        # Máscara para filtrar apenas as linhas que contêm códigos válidos das categorias (não textuais)
        mask = pd.to_numeric(df_excel['Código da categoria'], errors='coerce').notna()

        # Filtra pela máscara e pelas colunas "Código da variável", "Código da categoria" e "Descrição da categoria"
        df_excel = df_excel.loc[mask, ['Código da variável', 'Código da categoria', 'Descrição da categoria']]
        df_excel['Código da categoria'] = pd.to_numeric(df_excel['Código da categoria'], errors='coerce').astype('Int64')

        # Cria o dicionário aninhado para a substituição de valores das colunas
        map_values = {}
        for var_code, group in df_excel.groupby('Código da variável'):
            map_values[str(var_code).strip()] = group.set_index('Código da categoria')['Descrição da categoria'].to_dict()

    except FileNotFoundError:
        print(f"Erro: O arquivo '{path_excel}' não foi encontrado.")
        exit()
    
    df_to_return = df_to_translate.replace(map_values)
    df_to_return.columns = columns_names

    return df_to_return

In [17]:
path_excel = r"data\raw\panorama_politico_06_2024\DATASEN BR 2024 06 BAROMETRO - DICIONARIO DE DADOS.xlsx"
columns_names = [ 'estado', 'motivo_compart_fnews', 'identificar_fnews', 'empresas_impedir_fnews', 'importancia_controle_fnews',
    'petrobras_combustiveis', 'uso_maconha', 'cotas_universidades', 'direito_aborto', 'pena_morte', 'posse_armas', 'confianca_urnas',
    'satisfacao_democracia', 'regime_governo', 'sexo', 'cor_raca', 'religiao', 'renda_familiar', 'escolaridade', 'regiao', 
    'faixa_etaria', 'posicionamento_politico']

df_text = map_df_to_text(df, path_excel, columns_names)